<a href="https://colab.research.google.com/github/Yen-Nhi-23/TTNT/blob/main/Thuat_toan_minmax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Thuật toán min_max

In [11]:
import copy
import math
import numpy

# Định nghĩa các hằng số đại diện cho quân cờ
X = "X"
O = "O"
EMPTY = None
user = None
ai = None

# Khởi tạo bàn cờ rỗng kích thước 3x3
def initial_state():
  return [[EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY],
          [EMPTY, EMPTY, EMPTY]]

# xác định xem hiện tại đang là lượt của ai (X hay O)
# logic: Đếm số lượng quân cờ trên bàn
# Nếu số lượng X > O thì O đi. Nếu bằng nhau lượt đầu tiên đánh
def player(board):
  count = 0
  for i in board:
      for j in i:
          if j:
              count += 1
  # này mặc định là X đi trước
  if count % 2 != 0:
      return O
  return X

# Trả về một tập hợp các nước đi hợp lệ.
def actions(board):
  res = set()
  board_len = len(board)
  for i in range(board_len):
      for j in range(board_len):
          if board[i][j] == EMPTY:
              res.add((i, j))
  return res

# trả về trạng thái bàn cờ mới sau khi thực hiện hành động action
def result(board, action):
  curr_player = player(board) # Xác định người chơi hiện tại X hay O
  result_board = copy.deepcopy(board) # phải lưu lại trạng thái bàn cờ gốc
  (i, j) = action
  result_board[i][j] = curr_player
  return result_board

# kiểm tra thắng theo hàng ngang
def get_horizontal_winner(board):
  winner_val = None
  board_len = len(board)
  for i in range(board_len):
      winner_val = board[i][0]
      for j in range(board_len):
        if board[i][j] != winner_val:
          winner_val = None
      if winner_val:
        return winner_val
  return winner_val

# kiểm tra theo cột
def get_vertical_winner(board):
  winner_val = None
  board_len = len(board)
  for i in range(board_len):
      winner_val = board[0][i]
      for j in range(board_len):
          if board[j][i] != winner_val:
              winner_val = None
      if winner_val:
          return winner_val
  return winner_val

# kiểm tra thắng theo 2 chéo.
def get_diagonal_winner(board):
  winner_val = None
  board_len = len(board)
  # Chéo chính
  winner_val = board[0][0]
  for i in range(board_len):
      if board[i][i] != winner_val:
          winner_val = None
  if winner_val:
      return winner_val

  # Chéo phụ
  winner_val = board[0][board_len - 1]
  for i in range(board_len):
      j = board_len - 1 - i
      if board[i][j] != winner_val:
          winner_val = None
  return winner_val

# trả về người thắng cuộc nếu None chưa ai thắng
def winner(board):
  winner_val = get_horizontal_winner(board) or get_vertical_winner(board) or get_diagonal_winner(board) or None
  return winner_val

# Kết thúc: Kiểm tra xem game đã kết thúc chưa.trả về True nếu đã có hoặc Hòa
# ngược lại trả False
def terminal(board):
  if winner(board) != None:
      return True
  for i in board:
      for j in i:
          if j == EMPTY:
              return False
  return True

# hàm tiện ích tính điểm cho người chơi
# X thắng: +1 điểm
# O thắng: -1 điểm
# Hòa: 0 điểm
def utility(board):
  winner_val = winner(board)
  if winner_val == X:
      return 1
  elif winner_val == O:
      return -1
  return 0

# Hàm tìm giá trị nhỏ nhất (Dành cho lượt của O).
# O sẽ thử tất cả các nước đi và chọn nước đi dẫn đến kết quả tồi tệ nhất cho X (utility thấp nhất).
def minValue(state):
  if terminal(state):
      return utility(state)
  v = math.inf
  for action in actions(state):
      v = min(v, maxValue(result(state, action)))
  return v

# Hàm tìm giá trị lớn nhất (Dành cho lượt của X).
# X sẽ thử tất cả các nước đi và chọn nước đi dẫn đến kết quả tốt nhất cho mình (utility cao nhất).
def maxValue(state):
  if terminal(state):
      return utility(state)
  v = -math.inf
  for action in actions(state):
      v = max(v, minValue(result(state, action)))
  return v

# Hàm chính của AI. Quyết định nước đi tối ưu nhất từ trạng thái hiện tại.
def minimax(board):
  current_player = player(board)
  if current_player == X:
      curr_max = -math.inf
      move = None
      for action in actions(board):
          # X là max player, nên tìm nước đi dẫn đến giá trị min của O lớn nhất
          check = minValue(result(board, action))
          if check > curr_max:
              curr_max = check
              move = action
  else:
      curr_min = math.inf
      move = None
      for action in actions(board):
          # O là min player
          check = maxValue(result(board, action))
          if check < curr_min:
              curr_min = check
              move = action
  return move

if __name__ == "__main__":
  board = initial_state()
  ai_turn = False
  print("Chọn người chơi (X đi trước, O đi sau)")
  user = input("Bạn chọn X hay O? ").upper()

  if user == "X":
      ai = 'O'
      # User đi trước
  else:
      ai = "X"
      ai_turn = True # AI (X) đi trước

  while True:
      game_over = terminal(board)
      playr = player(board)

      if game_over:
          winner_p = winner(board)
          if winner_p is None:
              print("Game Over: Tie.")
          else:
              print(f"Game Over: {winner_p} wins.")
          print(numpy.array(board))
          break
      else:
          # Lượt AI
          if (playr == ai):
              print("AI is thinking...")
              move = minimax(board)
              board = result(board, move)
              print(numpy.array(board))

          # Lượt Người chơi
          else:
              print(f"Lượt của bạn ({user})")
              print("Nhập tọa độ (row, col): 0-2")
              try:
                  i = int(input("Row: "))
                  j = int(input("Col: "))
                  if board[i][j] == EMPTY:
                      board = result(board, (i, j))
                      print(numpy.array(board))
                  else:
                      print("Ô này đã có người đánh!")
              except (ValueError, IndexError):
                  print("Tọa độ không hợp lệ, vui lòng nhập lại.")

Chọn người chơi (X đi trước, O đi sau)
Bạn chọn X hay O? X
Lượt của bạn (X)
Nhập tọa độ (row, col): 0-2
Row: 0
Col: 0
[['X' None None]
 [None None None]
 [None None None]]
AI is thinking...
[['X' None None]
 [None 'O' None]
 [None None None]]
Lượt của bạn (X)
Nhập tọa độ (row, col): 0-2
Row: 
Tọa độ không hợp lệ, vui lòng nhập lại.
Lượt của bạn (X)
Nhập tọa độ (row, col): 0-2
Row: 0
Col: 1
[['X' 'X' None]
 [None 'O' None]
 [None None None]]
AI is thinking...
[['X' 'X' 'O']
 [None 'O' None]
 [None None None]]
Lượt của bạn (X)
Nhập tọa độ (row, col): 0-2
Row: 0
Col: 2
Ô này đã có người đánh!
Lượt của bạn (X)
Nhập tọa độ (row, col): 0-2
Row: 0
Col: 1
Ô này đã có người đánh!
Lượt của bạn (X)
Nhập tọa độ (row, col): 0-2
Row: 1
Col: 1
Ô này đã có người đánh!
Lượt của bạn (X)
Nhập tọa độ (row, col): 0-2
Row: 1
Col: 0
[['X' 'X' 'O']
 ['X' 'O' None]
 [None None None]]
AI is thinking...
[['X' 'X' 'O']
 ['X' 'O' None]
 ['O' None None]]
Game Over: O wins.
[['X' 'X' 'O']
 ['X' 'O' None]
 ['O' None 